In [ ]:
import pandas as pd

df = pd.read_csv("DATASET-balanced.csv")

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

print(X.head(10))
print(y.head(10))

   chroma_stft       rms  spectral_centroid  spectral_bandwidth      rolloff  \
0     0.338055  0.027948        2842.948867         4322.916759  6570.586186   
1     0.443766  0.037838        2336.129597         3445.777044  3764.949874   
2     0.302528  0.056578        2692.988386         2861.133180  4716.610271   
3     0.319933  0.031504        2241.665382         3503.766175  3798.641521   
4     0.420055  0.016158        2526.069123         3102.659519  5025.077899   
5     0.442880  0.012317        3952.880304         3702.717829  7104.089991   
6     0.453897  0.021782        4178.072150         3698.644769  7508.242075   
7     0.474154  0.011107        3993.039753         3948.154333  7872.563956   
8     0.602690  0.000970        3815.431438         3992.517515  6887.564689   
9     0.453962  0.017612        2894.560788         3435.434131  5663.232422   

   zero_crossing_rate       mfcc1       mfcc2      mfcc3      mfcc4  ...  \
0            0.041050 -462.169586   90.3112

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(y)
y = lb.transform(y)
y = y.ravel()
print(y)

[0 0 0 ... 1 1 1]


In [ ]:
#from sklearn.neighbors import KNeighborsClassifier
#model = KNeighborsClassifier(leaf_size=35 ,n_neighbors=15)


#from sklearn.model_selection import KFold
#kf = KFold(n_splits=5,  shuffle=True, random_state=1)

#print(model)
#print("KFold splits: " + str(kf.get_n_splits(X)))

KNeighborsClassifier(leaf_size=35, n_neighbors=15)
KFold splits: 5


In [ ]:
#from sklearn.svm import SVC

# إنشاء النموذج
#model = SVC(kernel='rbf',C=60,degree=2)
#from sklearn.linear_model import LogisticRegression

# إنشاء النموذج
#model = LogisticRegression()
#from sklearn.tree import DecisionTreeClassifier

# إنشاء النموذج
#model = DecisionTreeClassifier()
#from sklearn.ensemble import GradientBoostingClassifier
# إنشاء النموذج
#model = GradientBoostingClassifier()
#from sklearn.linear_model import SGDClassifier
#model= SGDClassifier()
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier(max_depth=5,min_samples_leaf=2)
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,  shuffle=True, random_state=1)

print(model)
print("KFold splits: " + str(kf.get_n_splits(X)))

HistGradientBoostingClassifier(max_depth=5, min_samples_leaf=2)
KFold splits: 5


In [ ]:
HistGradientBoostingClassifier?

In [ ]:

import time

import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

acc_score = []
prec_score = []
rec_score = []
f1s = []
MCCs = []
ROCareas = []
acc_train=[]
acc_test=[]
start = time.time()
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
    acctrain=model.score(X_train,y_train)
    acc_train.append(acctrain)
    acctest=model.score(X_test,y_test)
    acc_test.append(acctest)
    prec = precision_score(y_test , pred_values, average="binary", pos_label=1)
    prec_score.append(prec)

    rec = recall_score(y_test , pred_values, average="binary", pos_label=1)
    rec_score.append(rec)

    f1 = f1_score(y_test , pred_values, average="binary", pos_label=1)
    f1s.append(f1)

    mcc = matthews_corrcoef(y_test , pred_values)
    MCCs.append(mcc)

    roc = roc_auc_score(y_test , pred_values)
    ROCareas.append(roc)

end = time.time()
timeTaken = (end - start)
print("Model trained in: " + str( round(timeTaken, 2) ) + " seconds.")

Model trained in: 3.38 seconds.


In [ ]:
print("Mean results and (std.):\n")
print("Accuracy: " + str( round(np.mean(acc_score)*100, 3) ) + "% (" + str( round(np.std(acc_score)*100, 3) ) + ")\n")
print("Precision: " + str( round(np.mean(prec_score), 3) ) + " (" + str( round(np.std(prec_score), 3) ) + ")")
print("Recall: " + str( round(np.mean(rec_score), 3) ) + " (" + str( round(np.std(rec_score), 3) ) + ")")
print("F1-Score: " + str( round(np.mean(f1s), 3) ) + " (" + str( round(np.std(f1s), 3) ) + ")")
print("MCC: " + str( round(np.mean(MCCs), 3) ) + " (" + str( round(np.std(MCCs), 3) ) + ")")
print("ROC AUC: " + str( round(np.mean(ROCareas), 3) ) + " (" + str( round(np.std(ROCareas), 3) ) + ")")
print(acc_train)
print(acc_test)

Mean results and (std.):

Accuracy: 98.523% (0.287)

Precision: 0.99 (0.005)
Recall: 0.98 (0.004)
F1-Score: 0.985 (0.003)
MCC: 0.971 (0.006)
ROC AUC: 0.985 (0.003)
[0.9984079813203142, 0.9987263850562513, 0.9987263850562513, 0.999151013477661, 0.9989387668470763]
[0.9859932088285229, 0.9876910016977929, 0.980899830220713, 0.9830148619957537, 0.9885350318471338]


In [ ]:
import librosa
import numpy as np


def extract_features(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=12)
    chroma_stft = chroma_stft[:1]
    rms = librosa.feature.rms(y=y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    mfccs = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=20 )

    # Concatenate features along the correct axis
    features = np.concatenate([chroma_stft, rms, spectral_centroid, spectral_bandwidth, rolloff, zero_crossing_rate,mfccs], axis=0)

    # Take the mean along the feature axis
    features = np.mean(features, axis=1)

    return features

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

def convert_mp4_to_mp3(input_file, output_file):
    # Load the video clip
    video_clip = VideoFileClip(input_file)

    # Extract audio from the video clip
    audio_clip = video_clip.audio

    # Save the audio clip as an MP3 file
    audio_clip.write_audiofile(output_file)

    # Close the video and audio clips
    video_clip.close()
    audio_clip.close()

#if name == "main":
    input_file_path = "/content/video_2023-12-23_16-20-27.mp4"  # Replace with the path to your input MP4 file
    output_file_path = "/content/output_audio2.mp3"  # Replace with the desired output MP3 file path

    convert_mp4_to_mp3(input_file_path, output_file_path)

In [ ]:
extracted_features = extract_features('output_audio10.mp3')
extracted_features = extracted_features.reshape(1, -1)

In [ ]:
extracted_features.shape

(1, 26)

In [ ]:
extracted_features_df = pd.DataFrame(extracted_features, columns=X_train.columns)

In [ ]:
extracted_features_df

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.515619,0.077948,1006.974668,1311.727349,1799.434952,0.025821,-307.386853,174.892614,22.935383,4.445701,...,-5.534829,-3.544725,-4.556477,-4.881964,-4.610014,-5.559209,-6.55907,-5.169925,-2.462851,-2.129107


In [ ]:
prediction = model.predict(extracted_features_df)

In [ ]:
lb.inverse_transform(prediction)

array(['REAL'], dtype='<U4')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Make predictions on the test set
y_pred1 = model.predict(X_test)

# Convert probabilities to class labels
y_pred_classes = (y_pred1 > 0.5).astype(int)

# Create a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))


Confusion Matrix:
[[1082   80]
 [  68 1125]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      1162
           1       0.93      0.94      0.94      1193

    accuracy                           0.94      2355
   macro avg       0.94      0.94      0.94      2355
weighted avg       0.94      0.94      0.94      2355

